In [1]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# general eWC
import ewatercycle
import ewatercycle.forcing
import ewatercycle.models
import ewatercycle.analysis

In [3]:
from copy import copy

#### set up paths

In [4]:
path = Path.cwd()
forcing_path = path / "Forcing"

In [5]:
import xarray as xr

In [9]:
type(np.datetime64('1981-01-01T00:00:00'))

numpy.datetime64

In [14]:
start_time="1981-01-01T00:00:00Z"
end_time="1981-03-01T00:00:00Z"

In [15]:

def crop_ds(ds: xr.Dataset, start_time: str, end_time: str) -> xr.Dataset:
    """Crops dataset based on time."""
    get_time(start_time), get_time(end_time)  # if utc, remove Z to parse to np.dt64
    start, end = np.datetime64(start_time[:-1]), np.datetime64(end_time[:-1])
    return ds.isel(
        time=(ds["time"].to_numpy() >= start) & (ds["time"].to_numpy() <= end)
    )


In [10]:
ds = xr.open_dataset(forcing_path/"test_caravan_file.nc")

In [17]:
from ewatercycle.util import get_time

In [18]:
crop_ds(ds, start_time, end_time)

<xarray.Dataset>
Dimensions:                             (time: 60, basin_id: 2)
Coordinates:
  * time                                (time) datetime64[ns] 1981-01-01 ... ...
  * basin_id                            (basin_id) |S64 b'camels_01022500' b'...
Data variables: (12/55)
    timezone                            (basin_id) |S64 ...
    name                                (basin_id) |S64 ...
    country                             (basin_id) |S64 ...
    lat                                 (basin_id) float64 ...
    lon                                 (basin_id) float64 ...
    area                                (basin_id) float64 ...
    ...                                  ...
    volumetric_soil_water_layer_2_max   (basin_id, time) float32 ...
    volumetric_soil_water_layer_3_max   (basin_id, time) float32 ...
    volumetric_soil_water_layer_4_max   (basin_id, time) float32 ...
    total_precipitation_sum             (basin_id, time) float32 ...
    potential_evaporation_sum           (basin_id, time) float32 ...
    streamflow                          (basin_id, time) float32 ...
Attributes:
    history:  Wed Mar 27 16:11:00 2024: /usr/bin/ncap2 -s time=double(time) -...
    NCO:      netCDF Operators version 5.0.6 (Homepage = http://nco.sf.net, C...

In [6]:
from ewatercycle.util import (
    find_closest_point,
    fit_extents_to_grid,
    get_time,
    merge_esvmaltool_datasets,
    reindex,
    to_absolute_path,
)

## Test fail like in the unit test

In [7]:
def test_merge_esmvaltool_datasets():
    files = list((forcing_path).glob("OBS6_ERA5*.nc"))
    datasets = [xr.open_dataset(file) for file in files]
    ds = merge_esvmaltool_datasets(datasets)
    for var in ["tas", "pr", "rsds"]:
        assert not ds[var].mean(dim=["lat", "lon"]).isnull().any("time")

    assert "height" in ds["tas"].attrs


In [8]:
test_merge_esmvaltool_datasets()

AssertionError: 

In [9]:
files = list((forcing_path).glob("OBS6_ERA5*.nc"))
datasets = [xr.open_dataset(file) for file in files]
ds = merge_esvmaltool_datasets(datasets)

In [10]:
ds['tas']

<xarray.DataArray 'tas' (time: 30, lat: 25, lon: 33)> Size: 99kB
[24750 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 240B 2000-01-01T12:00:00 ... 2000-01-30T12...
  * lon      (lon) float64 264B 4.0 4.25 4.5 4.75 5.0 ... 11.25 11.5 11.75 12.0
  * lat      (lat) float64 200B 46.25 46.5 46.75 47.0 ... 51.5 51.75 52.0 52.25
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_methods:   day_of_year: year: mean

## But if we load it as in the code

In [11]:
def merge_esvmaltool_datasets(datasets: list[xr.Dataset]) -> xr.Dataset:
    """Merge the separate output datasets from an ESMValTool recipe into one dataset.

    ESMValTool has bad management of floating point precision in coordinates. Every
    CMORized file can have different rounding errors in the values of its coordinates.
    This will prevent easy merging with xarray's open_mfdataset, or combine_by_coords.
    By rounding to the 7th decimal place, more than sufficient precision is preserved
    ('waldo-on-a-page' precision)[1], while solving the floating point inprecision
    issue.

    References:
        [1] Randall Monroe, 2019. xkcd: Coordinate Precision. https://xkcd.com/2170/
    """
    TOLERANCE = 1e-7
    datasets = copy(datasets)

    # First check that the coordinates all line up before merging.
    for coord in ["lat", "lon"]:
        coords = [ds[coord].to_numpy() for ds in datasets]
        if len(set([c.size for c in coords])) > 1:
            msg = f"The coordinate '{coord}' is not of the same size in every dataset."
            raise ValueError(msg)
        all_coords = np.array(coords)
        if not np.all((all_coords - all_coords.mean(axis=0)) < TOLERANCE):
            msg = f"Coordinate {coord} deviates more than {TOLERANCE}. Merging failed."
            raise ValueError(msg)

    removed = {
        "lat_bnds": False,
        "lon_bnds": False,
    }
    for i in range(len(datasets)):
        # Bounds are not aligned, and can be missing in derived vars,
        #  so we remove all except the first lat/lon bounds we encounter.
        for coord in ["lat_bnds", "lon_bnds"]:
            if coord in datasets[i]:
                if removed[coord]:
                    datasets[i] = datasets[i].drop_vars(coord)
                removed[coord] = True

        # xr.align doesn't work for lumped forcing. this works for both lumped and dist:
        for coord in ["lat", "lon"]:
            datasets[i][coord] = datasets[0][coord]

        # the time coordinates are messed up for some files, see:
        #   https://github.com/eWaterCycle/infra/issues/157
        #   the following is a workaround.
        if "time_bnds" in datasets[i] and xr.infer_freq(datasets[i]["time"]) == "D":
            datasets[i]["time"] = datasets[i]["time_bnds"].isel(
                bnds=0
            ) + np.timedelta64(12, "h")
            datasets[i] = datasets[i].drop_vars("time_bnds")

        # A "height" coordinate can be present, which will result in conflicts.
        #   Instead, we move it to the variable's attributes.
        if "height" in datasets[i].variables:
            data_vars = list(datasets[i].data_vars)
            if "time_bnds" in data_vars:
                data_vars.remove("time_bnds")
            # var = data_vars[-1]
            var = "tas"
            datasets[i][var].attrs.update(
                {
                    "height": float(datasets[i]["height"]),
                    "height_units": datasets[i]["height"].attrs["units"],
                }
            )
            datasets[i] = datasets[i].drop_vars(("height",))

    return xr.combine_by_coords(datasets, combine_attrs="drop_conflicts")  # type: ignore[return-value]


## Suddendly it does work

In [12]:
ds = merge_esvmaltool_datasets(datasets)

In [13]:
ds['tas']

<xarray.DataArray 'tas' (time: 30, lat: 25, lon: 33)> Size: 99kB
[24750 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 240B 2000-01-01T12:00:00 ... 2000-01-30T12...
  * lon      (lon) float64 264B 4.0 4.25 4.5 4.75 5.0 ... 11.25 11.5 11.75 12.0
  * lat      (lat) float64 200B 46.25 46.5 46.75 47.0 ... 51.5 51.75 52.0 52.25
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_methods:   day_of_year: year: mean
    height:         2.0
    height_units:   m

In [14]:
ds['tas'].attrs.update()

In [15]:
datasets = []
i = 0
datasets.append(xr.open_dataset(files[i]))

In [16]:
'height' in datasets[0].variables.keys()

True

In [17]:
removed = {
    "lat_bnds": False,
    "lon_bnds": False,
}
for i in range(len(datasets)):
    # Bounds are not aligned, and can be missing in derived vars,
    #  so we remove all except the first lat/lon bounds we encounter.
    for coord in ["lat_bnds", "lon_bnds"]:
        if coord in datasets[i]:
            if removed[coord]:
                datasets[i] = datasets[i].drop_vars(coord)
            removed[coord] = True

    # xr.align doesn't work for lumped forcing. this works for both lumped and dist:
    for coord in ["lat", "lon"]:
        datasets[i][coord] = datasets[0][coord]

    # the time coordinates are messed up for some files, see:
    #   https://github.com/eWaterCycle/infra/issues/157
    #   the following is a workaround.
    if "time_bnds" in datasets[i] and xr.infer_freq(datasets[i]["time"]) == "D":
        datasets[i]["time"] = datasets[i]["time_bnds"].isel(
            bnds=0
        ) + np.timedelta64(12, "h")
        datasets[i] = datasets[i].drop_vars("time_bnds")

    # A "height" coordinate can be present, which will result in conflicts.
    #   Instead, we move it to the variable's attributes.
    if "height" in datasets[i].variables:
        data_vars = list(datasets[i].data_vars)
        if "time_bnds" in data_vars:
            data_vars.remove("time_bnds")
        # var = data_vars[-1]
        var = "tas"
        datasets[i][var].attrs.update(
            {
                "height": float(datasets[i]["height"]),
                "height_units": datasets[i]["height"].attrs["units"],
            }
        )
        datasets[i] = datasets[i].drop_vars(("height",))

In [18]:
datasets[0]['tas']

<xarray.DataArray 'tas' (time: 30, lat: 25, lon: 33)> Size: 99kB
[24750 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 240B 2000-01-01T12:00:00 ... 2000-01-30T12...
  * lon      (lon) float64 264B 4.0 4.25 4.5 4.75 5.0 ... 11.25 11.5 11.75 12.0
  * lat      (lat) float64 200B 46.25 46.5 46.75 47.0 ... 51.5 51.75 52.0 52.25
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_methods:   day_of_year: year: mean
    height:         2.0
    height_units:   m

In [19]:
datasets[i]['tas'].attrs.update({"Height":2})

In [20]:
datasets[i]

<xarray.Dataset> Size: 101kB
Dimensions:   (time: 30, lon: 33, bnds: 2, lat: 25)
Coordinates:
  * time      (time) datetime64[ns] 240B 2000-01-01T12:00:00 ... 2000-01-30T1...
  * lon       (lon) float64 264B 4.0 4.25 4.5 4.75 5.0 ... 11.25 11.5 11.75 12.0
  * lat       (lat) float64 200B 46.25 46.5 46.75 47.0 ... 51.5 51.75 52.0 52.25
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (lon, bnds) float64 528B ...
    lat_bnds  (lat, bnds) float64 400B ...
    tas       (time, lat, lon) float32 99kB ...
Attributes:
    CDI:          Climate Data Interface version 2.3.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.7
    software:     Created with ESMValTool v2.10.0
    caption:      Forcings for generic hydrological model
    history:      Wed Mar 20 10:42:59 2024: cdo seltimestep,1/30 OBS6_ERA5_re...
    CDO:          Climate Data Operators version 2.3.0 (https://mpimet.mpg.de...

In [21]:
datasets[i]['tas']

<xarray.DataArray 'tas' (time: 30, lat: 25, lon: 33)> Size: 99kB
[24750 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 240B 2000-01-01T12:00:00 ... 2000-01-30T12...
  * lon      (lon) float64 264B 4.0 4.25 4.5 4.75 5.0 ... 11.25 11.5 11.75 12.0
  * lat      (lat) float64 200B 46.25 46.5 46.75 47.0 ... 51.5 51.75 52.0 52.25
Attributes:
    standard_name:  air_temperature
    long_name:      Near-Surface Air Temperature
    units:          K
    cell_methods:   day_of_year: year: mean
    height:         2.0
    height_units:   m
    Height:         2

In [22]:
if "height" in datasets[i].variables:
    data_vars = list(datasets[i].data_vars)
    if "time_bnds" in data_vars:
        data_vars.remove("time_bnds")
    var = data_vars[-1]
    datasets[i][var].attrs.update(
        {
            "height": float(datasets[i]["height"]),
            "height_units": datasets[i]["height"].attrs["units"],
        }
    )
    print( datasets[i][var].attrs)
    datasets[i] = datasets[i].drop_vars(("height",))


In [23]:
datasets[i].data_vars

Data variables:
    lon_bnds  (lon, bnds) float64 528B ...
    lat_bnds  (lat, bnds) float64 400B ...
    tas       (time, lat, lon) float32 99kB ...

In [24]:
ds

<xarray.Dataset> Size: 299kB
Dimensions:   (time: 30, lon: 33, bnds: 2, lat: 25)
Coordinates:
  * time      (time) datetime64[ns] 240B 2000-01-01T12:00:00 ... 2000-01-30T1...
  * lon       (lon) float64 264B 4.0 4.25 4.5 4.75 5.0 ... 11.25 11.5 11.75 12.0
  * lat       (lat) float64 200B 46.25 46.5 46.75 47.0 ... 51.5 51.75 52.0 52.25
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (lon, bnds) float64 528B ...
    lat_bnds  (lat, bnds) float64 400B ...
    tas       (time, lat, lon) float32 99kB ...
    pr        (time, lat, lon) float32 99kB ...
    rsds      (time, lat, lon) float32 99kB ...
Attributes:
    CDI:          Climate Data Interface version 2.3.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.7
    software:     Created with ESMValTool v2.10.0
    caption:      Forcings for generic hydrological model
    CDO:          Climate Data Operators version 2.3.0 (https://mpimet.mpg.de...
    positive:     down

In [25]:
def test_merge_esmvaltool_datasets():
    files = list((forcing_path).glob("OBS6_ERA5*.nc"))
    datasets = [xr.open_dataset(file) for file in files]
    ds = merge_esvmaltool_datasets(datasets)
    for var in ["tas", "pr", "rsds"]:
        assert not ds[var].mean(dim=["lat", "lon"]).isnull().any("time")

    assert "height" in ds["tas"].attrs

In [26]:
test_merge_esmvaltool_datasets()

# fix string match to name in use

In [10]:
import netCDF4 as ncf
print(ncf.__version__)
print(ncf.getlibversion())

f = ncf.Dataset('test12122.nc', 'w')
f.createDimension('test', 6)
var = f.createVariable('test', 'f', ('test',))
var[:] = 0
f.setncattr('NAME', 1)


1.6.5
4.9.2 of Dec 10 2023 17:23:27 $


PermissionError: [Errno 13] Permission denied: 'test12122.nc'

In [11]:
attrs = {"NAME":1,"test":2}

In [15]:
if "NAME" in attrs:
    rename_val  = attrs["NAME"]
    attrs.pop("NAME")
    attrs['name'] = rename_val

In [16]:
attrs

{'test': 2, 'name': 1}